# Introduction to Unsupervised Learning 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')

%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

#### Grab and install a custom package that we will use for plotting

In [ ]:
%pip install git+https://github.com/agdelma/ml4s.git#egg=ml4s

import ml4s
ml4s.set_css_style('./include/bootstrap.css')

## Until Now

### Supervised Learning

- We have considered learning tasks (regression, classification) where there was labelled data and we could train a neural network to "learn" these features of the data. 
- This is incredibly useful, but many problems in the physical sciences aren't necessarily about prediciton. We usually want to **learn** something about the underlying distribution that generated the observation. 
- It is not useful in physics to make good predictions with the wrong model. 

## Now

- Our first foray into unsupervised learning, a large and exciting field.
- Concerned with discovering structure in unalabelled data.
- We will begin with dimensional reduction.

## Dimensional Reduction

Most of our data sets live in a very high dimension (*e.g.* consider a $30\times 30$ Ising model, configurations live in 900 dimensional space of binary numbers!)  Our main goal will be to project (or embed) high dimensional data (observations) into a lower dimensional space where it can be better analyzed / understood.   

This subspace is called the **latent space**.

Let's consider a very simple example of points in two spatial dimensions.

<!--
N = 2000
x = np.random.normal(loc=[0,0],scale=[1,0.4], size=[N,2])

θ = np.radians(35)
R = np.array([[np.cos(θ), -np.sin(θ)], [np.sin(θ), np.sin(θ)]])
for i in range(N):
    x[i] = R @ x[i]
np.savetxt('../data/scatter_2d_pca.dat',x)
-->

In [ ]:
x = np.loadtxt('./data/scatter_2d_pca.dat')
plt.scatter(x[:,0],x[:,1], s=1)
plt.axis('equal')
plt.xticks([])
plt.yticks([]);

### Principal Component Analysis (PCA)

The goal of PCA is to identify the directions of largest variance with the intuition that these correspond to **signal**, while any orthogonal spread in the data is due to **noise**.  It has an equivalent definition in terms of the linear projection that minimizes the average projection cost (mean squared deviation between original vector and its deviation).


Consider the set of observations $\{ \boldsymbol{x}^{(n)} \}_{n=1}^{N}$ where, unlike the supervised case,  we do not have any associated targets or *labels* $y^{(n)}$.  Each $\boldsymbol{x}^{(n)} \in \mathbb{R}^D$ lives in a D-dimensional feature space.  

Without loss of generality, we assume that the mean of the data set is zero: 

\begin{equation}
\langle \boldsymbol{x} \rangle = \frac{1}{N} \sum_{n=1}^{N} \boldsymbol{x}^{(n)} = (0,\dots,0).
\end{equation}

If this is not the case, we can simply subtract the mean from each data point.  

**Goal:** project the data onto a latent space having dimensionality $M < D$.

To begin, let's project onto a 1-dimenisional space, i.e. $M=1$.  The direction of this dimension (i.e. its unit vector) can be described by a single vector, $\boldsymbol{v}_1$, which we assume has unit norm: $\boldsymbol{v}_1 \cdot \boldsymbol{v}_1 = 1$.  We then project each data point, $\boldsymbol{x}^{(n)}$ onto $\boldsymbol{v}_1$ via $\boldsymbol{v}_1^\top \boldsymbol{x}^{(n)}$. 

The variance of the resulting data set is given by:

\begin{align}
\sigma^2 \equiv \bigg \langle \big\lvert\boldsymbol{v}_1^{\top} \boldsymbol{x}^{(n)} - \boldsymbol{v}_1^{\top}\underbrace{\langle  \boldsymbol{x} \rangle}_{0} \big\rvert^2 \bigg \rangle  & = \frac{1}{N-1} \sum_{n=1}^{N} \boldsymbol{v}_1^{\top}  \boldsymbol{x}^{(n)}  \boldsymbol{x}^{(n)\top}  \boldsymbol{v}_1 \\
& = \boldsymbol{v}_1^{\top} \Sigma(\mathbf{X}) \boldsymbol{v}_1
\end{align}

where we have defined

\begin{equation}
\Sigma(\mathbf{X}) = \frac{1}{N-1} \mathbf{X}^{\top}\mathbf{X}
\end{equation}

to be the $D \times D$ covariance matrix of the data design matrix: 

\begin{equation}
\mathbf{X} = \left( \begin{array}{cccc}
        x_{1}^{(1)} & x_{2}^{(1)} & \cdots & x_{D}^{(1)} \\
\vdots        &      \vdots    & \ddots & \vdots \\
        x_{1}^{(N)} & x_{2}^{(N)} & \cdots & x_{D}^{(N)} \\
\end{array}
\right)\, .
\end{equation}


We now want to maximize the variance subject to the constraint $\boldsymbol{v}_1 \cdot \boldsymbol{v}_1 = 1$ which we can do by adding a Lagrange multiplier $\lambda_1$:

\begin{align}
%\frac{\partial}{\partial \boldsymbol{v}_1^{\top}} 
\boldsymbol{\nabla}_{\boldsymbol{v}_1^{\top}} \left[\boldsymbol{v}_1^{\top} \Sigma(\mathbf{X}) \boldsymbol{v}_1 + \lambda_1 (1-\boldsymbol{v}_1^{\top} \cdot \boldsymbol{v}_1) \right] & = \Sigma(\mathbf{X}) \boldsymbol{v}_1 - \lambda_1 \boldsymbol{v}_1 = 0 \\
&\Rightarrow  \Sigma(\mathbf{X})\boldsymbol{v}_1 = \lambda_1 \boldsymbol{v}_1
\end{align}

which tells us that $\boldsymbol{v}_1$ is a left eigenvector of $\Sigma(\mathbf{X})$.  If we multiple on the left by $\boldsymbol{v}_1^{\top}$ and utilize the unit norm we find:

\begin{equation}
\boxed{\lambda_1 = \boldsymbol{v}_1^{\top} \Sigma(\mathbf{X}) \boldsymbol{v}_1.}
\end{equation}

Thus the variance will be maximized when $\boldsymbol{v}_1$ is chosen to be the the eigenvector with largest eignvalue $\lambda_1$.  It is known as the **first principle component**.  

Not surprisingly, we can determine additional principal components in an iterative fashion which maximizes the projected variance amongst all possible directions orthogonal to the ones already considered.  This identifies the $M$ principal components as the $M$ eigenvectors corresponding to the $M$ largest eigenvalues $\boldsymbol{v}_j$ of $\Sigma(\mathbf{X})$.

In practice, we simply diagonlize the symmetrix square matrix: $\Sigma(\mathbf{X}) = (N-1)^{-1}\mathbf{X}^{\top}\mathbf{X}$, the principle components are the eigenvalues $\lambda_j$ and we organize the eigenvectors into a column matrix $\boldsymbol{V}$.

We can do this with the `scipy.linalg` package.

In [ ]:
import scipy.linalg
N = x.shape[0]
x -= np.average(x,axis=0)
Σ = x.T @ x / (N-1)
λ,V = scipy.linalg.eigh(Σ)

<div class="span alert alert-warning">
    Note: <code>eigh()</code> returns eigenvalues in ascending order! We need to re-order to get the maximum eignvalues.
</div>

In [ ]:
λ = λ[::-1]
V = np.flip(V,axis=1)

print(f'λ = {λ}')
print(f'V = {V}')

The columns of $\mathbf{V}$ correspond to the *unit-vectors* that span our latent space.

\begin{equation}
\mathbf{V} = \left( \begin{array}{cc}
        v_{1,1} & v_{1,2} \\
        v_{2,1} & v_{2,2} \\
\end{array}
\right)\, .
\end{equation}

### Percentage of the Explained Variance

An important quantity is the *percentage of the explained variance* defined by:

\begin{equation}
\text{PCA-j} = \frac{\lambda_j}{\sum_{j=1}^{D} \lambda_j}
\end{equation}

We can plot the resulting axes defined by the principal components, or alternatively, can define a projection operator (matrix):

\begin{equation}
\boldsymbol{P} = \sum_{j=1}^M\boldsymbol{v}_j\boldsymbol{v}_j^\mathsf{T}
\end{equation}

to plot with respect to the new axes.


In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,4))

ax[0].scatter(x[:,0],x[:,1], s=1, alpha=0.5)

_x = np.linspace(-4,4,100)
_y = np.linspace(-0.5,0.5,100)
ax[0].plot(_x,V[1,0]/V[0,0]*_x, '-', color=colors[0], label=f'PCA-1 = {λ[0]/np.sum(λ):.2f}')
ax[0].plot(_y,V[1,1]/V[0,1]*_y, '-', color=colors[-2], label=f'PCA-2 = {λ[1]/np.sum(λ):.2f}')

ax[0].axis('equal')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].legend()

# perform the projection
px = x @ V

# this can alternatively be down via direct projection
direct = False
if direct:
    px = np.zeros_like(x)
    for n in range(x.shape[0]):
        for j in range(2):
            px[n,j] = np.dot(V[:,j],x[n,:])
    
ax[1].scatter(px[:,0],px[:,1], s=1, alpha=0.5, label=r'$\mathbf{X} \mathbf{V}$')
ax[1].set_xlabel(f'PCA-1 = {λ[0]/np.sum(λ):.2f}')
ax[1].set_ylabel(f'PCA-2 = {λ[1]/np.sum(λ):.2f}')
ax[1].axis('equal')
ax[1].legend()

fig.subplots_adjust(wspace=0.25)

### Sci-Kit Learn Implementation

Obviously this is useful enough that we don't have to code it up from scratch every time we want the principal components.  It has a convenient implementation in `sklearn`.

Our first step is to scale the data (so-called standard or z-scaling) such that:

\begin{equation}
\mathbf{z} = \frac{\mathbf{x}-\langle \mathbf{x} \rangle}{\sqrt{\langle \mathbf{x}^{\top}\mathbf{x} \rangle - \lvert \langle \mathbf{x} \rangle \rvert^2}} \, .
\end{equation}

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

model = PCA(n_components=2)

# perform the PCA on scaled_x and apply the dimensionality reduction
Z_pca = model.fit_transform(scaled_x)

λ = model.explained_variance_
V = model.components_.T
PCAj = model.explained_variance_ratio_

print(f'λ = {λ}')
print(f'V = {V}')
print(f'PCA-j = {PCAj}')

<div class="span alert alert-warning">
    Note: To be extra confusing, <code>model.components_.</code> returns the principal vectors as rows! I take the transpose above to compare with our eigenvectors above.
</div>

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(8,4))
ax[0].scatter(scaled_x[:,0],scaled_x[:,1], s=1, alpha=0.5)

_x = np.linspace(-4,4,100)
_y = np.linspace(-0.5,0.5,100)
ax[0].plot(_x,V[1,0]/V[0,0]*_x, '-', color=colors[0], label=f'PCA-1 = {λ[0]/np.sum(λ):.2f}')
ax[0].plot(_y,V[1,1]/V[0,1]*_y, '-', color=colors[-2], label=f'PCA-2 = {λ[1]/np.sum(λ):.2f}')
ax[0].axis('equal')
ax[0].legend()


ax[1].scatter(Z_pca[:,0],Z_pca[:,1], s=1, alpha=0.5, label=r'$\mathbf{X} \mathbf{V}$')
ax[1].set_xlabel(f'PCA-1 = {PCAj[0]:.2f}')
ax[1].set_ylabel(f'PCA-2 = {PCAj[1]:.2f}')
ax[1].axis('equal')
ax[1].legend()

fig.subplots_adjust(wspace=0.25)

<div class="span alert alert-success">
<h2>Use Case: Edge States in the Bernevig-Hughes-Zhang Model </h2>

In a recent <a href="https://journals.aps.org/prb/abstract/10.1103/PhysRevB.109.245115" title="Topological and magnetic properties of the interacting Bernevig-Hughes-Zhang model">paper</a> we used PCA to confirm the existence of a new topological phase in a simple model of a 2D Topological insulator. In this exercise you will confirm our analysis.
    
 1. Load the data corresponding to edge state electronic densities from a file.
 2. Perform a PCA analysis by copy/pasting code above and predict the dimension of the latent space.
 3. Plot a phase diagram where the color at each point corresponds to the first principal component.
</div>

### Load the data from disk

The data corresponds to the electronic densities at the edge of the BHZ model on a cylinder for different values of the mass $U$ and the interaction strength $m$.  The Cylinder is $4\times 4$ and we have two oribtals (*s* and *p*) thus each row of our file includes a $U$-value, $m$-value and 32 values of the orbital density. We have a different orbitial density $n_i$ for a total of 317 $(U,m)$ pairs.

<!--
Uidx = np.where(np.abs(U-5)<0.1)
midx = np.where(np.abs(m-1.75)<0.1)
np.intersect1d(Uidx,midx)
-->

In [ ]:
data = np.loadtxt("./data/bhz_orbital_density.dat")
U = data[:,0]
m = data[:,1]
ni = data[:,2:]

ni.shape

### Let's explore the raw DMRG oribtal occupancy data

In [ ]:
def plot_ni(ax,density,label=False):
    num_sites = density.shape[0] // 2
    label1,label2 = '',''
    if label:
        label1,label2 = '$n_i^s$','$n_i^p$'
        
    ax.plot(density[:num_sites], color=colors[1], lw=0.2, label=label1)
    ax.plot(density[num_sites:], color=colors[-2], lw=0.2, label=label1)

In [ ]:
fig,ax = plt.subplots()
plot_ni(ax,ni[0],label=True)

for j in range(1,ni.shape[0]):
    plot_ni(ax,ni[j])

plt.ylabel('orbital density')
plt.xlabel('site index')
plt.legend();

#### I've chosen three potentially interesting values

In [ ]:
plt.plot(ni[146,:], label=f'$U = {U[146]:.1f}, m={m[146]:.2f}$')
plt.plot(ni[282,:], label=f'$U = {U[282]:.1f}, m={m[282]:.2f}$')
plt.plot(ni[0,:], label=f'$U = {U[0]:.1f}, m={m[0]:.2f}$')

plt.legend(loc=(1.0,0.7))
plt.ylabel('orbital density')
plt.xlabel('site index')

### Perform the PCA Analysis using the code above

Above our input data was only 2-dimensional, here we have data that lives in $\mathbb{R}^{32}$.  A good first step is to consider only a small number of principal components and test the variance ratios.

In [ ]:
model = PCA(n_components=10)
Z_pca = model.fit_transform(ni)
PCAj = model.explained_variance_ratio_

### Investigate the explained variance ratios

In [ ]:
fig,ax = plt.subplots(1,1)
ax.plot(PCAj,'o')
ax.set_yscale('log')
ax.set_xlabel('component')
ax.set_ylabel('PCA-j')

<div class="span alert alert-warning">
    Almost everything is in the first component! We can see this by plotting the first two components against each other.
</div>

In [ ]:
fig,ax = plt.subplots(1,1)
ax.plot(Z_pca[:,0],Z_pca[:,1],'o')
ax.set_xlabel('component-1')
ax.set_ylabel('component-2')

## Investigate a phase diagram

For each value of $m$ and $U$ lets plots the the orbital density projected onto the first principal component.

In [ ]:
fig,ax = plt.subplots(1,1)
pd = ax.scatter(U,m,c=Z_pca[:,0], cmap='Spectral_r')
plt.colorbar(pd, label='component-1')
ax.set_xlabel('$U$')
ax.set_ylabel('$m$')

### Compare with the phase diagram in the paper computed from (expensive and conventional) analysis of magnetic and spectral properties

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://github.com/DelMaestroGroup/papers-code-interactingBHZmodel/blob/fe417edd2e5db387c5fd5635bdcba50d711b71c7/plots_and_scripts/Phase_Diagram_Plot/Magnetic_phase_diagram_Nx4_BHZ.png?raw=true", width=400)

<div class="span alert alert-info" role="alert">
    <h3>It Works!</h3>
</div>